# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Import the dataset with: `mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json`

This will correctly import the json containing the data that need to be processed through the Mongo NoSQL database. The `--drop` command ensures that any existing data are dropped prior to import. The `--jsonArray` parameter specifies the format of the source file. 

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)  # Using the default port

In [3]:
# confirm that our new database was created
mongo.list_database_names() #uk_food should be one of the listed databases

['admin',
 'autosaurus',
 'classDB',
 'config',
 'epa',
 'local',
 'met',
 'petsitly_marketing',
 'travel_db',
 'uk_food']

In [4]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [5]:
# review the collections in our new database
db.list_collection_names() # If the import was successful, 'establishments' should be the only valid collection in the database.

['establishments']

In [6]:
# review a document in the establishments collection
pprint(db.establishments.find_one(), indent=2) # This prints out a single document from within the collection. 
# The pprint functionality makes sure the document is easy to read.

{ 'AddressLine1': 'East Cliff Pavilion',
  'AddressLine2': 'Wear Bay Road',
  'AddressLine3': 'Folkestone',
  'AddressLine4': 'Kent',
  'BusinessName': 'The Pavilion',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  'ChangesByServerID': 0,
  'Distance': 4591.765489457773,
  'FHRSID': 1043695,
  'LocalAuthorityBusinessID': 'PI/000073616',
  'LocalAuthorityCode': '188',
  'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
  'LocalAuthorityName': 'Folkestone and Hythe',
  'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
  'NewRatingPending': False,
  'Phone': '',
  'PostCode': 'CT19 6BL',
  'RatingDate': '2018-04-04T00:00:00',
  'RatingKey': 'fhrs_5_en-gb',
  'RatingValue': 5,
  'RightToReply': '',
  'SchemeType': 'FHRS',
  '_id': ObjectId('662952a832d15c1a2cebd515'),
  'geocode': {'latitude': 51.083812, 'longitude': 1.195625},
  'links': [ { 'href': 'https://api.ratings.food.gov.uk/establishments/1043695',
               'rel': 'self'}],

In [7]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [8]:
# Create a dictionary for the new restaurant data
# These data were provided independently as a supplement to the UK Food Standards Agency.

new_restaurant = {
                'BusinessName':'Penang Flavours',
                'BusinessType':'Restaurant/Cafe/Canteen',
                'BusinessTypeID':'',
                'AddressLine1':'Penang Flavours',
                'AddressLine2':'146A Plumstead Rd',
                'AddressLine3':'London',
                'AddressLine4':'',
                'PostCode':'SE18 7DY',
                'Phone':'',
                'LocalAuthorityCode':'511',
                'LocalAuthorityName':'Greenwich',
                'LocalAuthorityWebSite':'http://www.royalgreenwich.gov.uk',
                'LocalAuthorityEmailAddress':'health@royalgreenwich.gov.uk',
                'scores':{
                            'Hygiene':'',
                            'Structural':'',
                            'ConfidenceInManagement':''
                        },
                'SchemeType':'FHRS',
                'geocode':{
                            'longitude':'0.08384000',
                            'latitude':'51.49014200'
                        },
                'RightToReply':'',
                'Distance':4623.9723280747176,
                'NewRatingPending':True
                }


In [9]:
# Insert the new restaurant into the collection
establishments.insert_one(new_restaurant)

In [10]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({'BusinessName':new_restaurant['BusinessName']})) # Shows that the restaurant now exists in the collection.

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RatingValue': None,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('662952bdb821c41d672c526b'),
 'geocode': {'latitude': 51.490142, 'longitude': 0.08384},
 'scores': {'ConfidenceInManagement': '', 'Hygiene': '', 'Structural': ''}}


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'} # Filters our find to look for just the 'Restaurant/Cafe/Canteen' types
fields = {'BusinessTypeID': 1, # Returns just the business type (to make sure we have the right query) and Type ID
          'BusinessType': 1
         }
establishments.find(query, fields)[0] # All records of a given business type have the same type ID, so we only need one of them.

{'_id': ObjectId('662952a832d15c1a2cebd515'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
# The data we were givin to add to the database for 'Penang Flavors' does not include
# a BusinessTypeID. Since we determined the correct value in the cell above, we can update
# that one document.
query = {'BusinessName': 'Penang Flavours'}
new_value = {'$set': {'BusinessTypeID': 1}}
establishments.update_one(query, new_value)

In [13]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one(
    {"BusinessName":new_restaurant["BusinessName"]},
    {'BusinessName': 1,
     'BusinessType': 1,
     'BusinessTypeID': 1}
    ), indent = 2
)

# When we look for Penang Flavours (using the business name in our original text), we find that it exists
# and now has the correct business type ID of '1'

{ 'BusinessName': 'Penang Flavours',
  'BusinessType': 'Restaurant/Cafe/Canteen',
  'BusinessTypeID': 1,
  '_id': ObjectId('662952bdb821c41d672c526b')}


4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
query = {'LocalAuthorityName': 'Dover'}
# I first wrote this using the syntax: len(list(establishments.find(query)))
# But the instructions said to use count_documents, so updated to the following:
print(f'Of the {establishments.count_documents({})} establishments, {establishments.count_documents(query)} are in Dover')

# There are 994 documents that list "Dover" as the local authority name. That is out of a total of 39780 establishments (so we can
# subtract in a moment and be sure that we have deleted the correct number of locations).

Of the 38787 establishments, 0 are in Dover


In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many(query) # We can re-use the query variable from the last cell to make our deletion.

In [16]:
# Check if any remaining documents include Dover
print(f'Of the {establishments.count_documents({})} remaining establishments, {establishments.count_documents(query)} are in Dover')

# There are no 'Dover' locations left, and if we add the original 994 'Dover' establishments to the 38786 remaining establishments,
# we get the 39780 establishments we found earlier. Looks like we successfully removed the correct establishments!

Of the 38787 remaining establishments, 0 are in Dover


In [17]:
# Check that other documents remain with 'find_one'
establishments.find_one({})

{'_id': ObjectId('662952a832d15c1a2cebd515'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.195625, 'latitude': 51.083812},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
query = {}
update = [{'$set': 
           {'geocode.longitude': 
                {'$toDecimal': '$geocode.longitude'},
            'geocode.latitude': 
                {'$toDecimal': '$geocode.latitude'}}}]
establishments.update_many(query, update)

# I chose to use 'Decimal' as it was requested, but 'Double' is ample precision for the actual data.

Use `update_many` to convert `RatingValue` to integer numbers.

In [19]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [20]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [{'$set': {'RatingValue': {'$toInt': '$RatingValue'}}}])

In [21]:
# Check that the coordinates and rating value are now numbers
establishments.find_one({})

# Since the RatingValue is no longer enclosed in quotes, it has been converted successfully to an integer.
# The listed longitudes and latitudes are now decimal numerals as can be seen from the label 'Decimal128'

{'_id': ObjectId('662952a832d15c1a2cebd515'),
 'FHRSID': 1043695,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000073616',
 'BusinessName': 'The Pavilion',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'East Cliff Pavilion',
 'AddressLine2': 'Wear Bay Road',
 'AddressLine3': 'Folkestone',
 'AddressLine4': 'Kent',
 'PostCode': 'CT19 6BL',
 'Phone': '',
 'RatingValue': 5,
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2018-04-04T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': Decimal128('1.19562500000000'),
  'latitude': Decimal128('51.0838120000000')},
 'RightToReply': '',
 'Distance': 4591.765489457773,
 'NewRatingPending': False,
 'meta': {'dataSour